In [1]:
# basic
import json
from tqdm import tqdm

# import langchain
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Vector Store
from langchain.vectorstores import Qdrant
from langchain.vectorstores.pgvector import PGVector

# import env
from config import Settings
settings = Settings()

In [3]:
with open("../data/baseDocs.json", "r") as f:
    # jsonファイルを読み込んで、辞書型に変換する
    data = json.load(f)
    base_docs = [Document(page_content=text["page_content"], metadata=text["metadata"]) for text in data]

base_docs

[Document(page_content='【キャリア自律とは】\nキャリア自律（Career Self-Reliance）は、1990年代半ばにアメリカで提唱され始めた概念です。アメリカの「キャリア・アクション・センター（CAC）」では、キャリア自律を「めまぐるしく変化する環境の中で、自らのキャリア構築と継続的学習に積極的に取り組む、生涯にわたるコミットメント」と定義しています。\n\nキャリア自律は、新しい気付きや自己変容に焦点を置いています。単に特定の分野においてスキルを高めていくことを目指すものではなく、環境が変化する中で、“自己概念は生涯にわたって変わり得る”との認識のもと、柔軟性を持って自分のキャリアを切り開いていくしセガ求められます。\n\n【自立と自律の違い】\n〇自立：他の助けを借りずに自力で物事を行う\n〇自律：他に強制されず自らをコントロールする\n\n慶應義塾大学名誉教授の花田光世さんは、これらの言葉を「個人」と結びつけたとき、次のような違いが生じるとしています。\n\n●自立の状態にある個人：自分の意見を持ち、自己の意見を主張できるが、個人の単なる自己視聴・満足で終わってしまっている状態\n●自律の状態にある個人：他者のニーズを把握し、それとの調整をはかりながら、自分自身の行動のコントロールを行い、自らを律しながら、自己実現を図ることのできる状態\n\nいま社会から求められているのは、自律の状態にある個人です。真のキャリア自律とは、自分自身のキャリアビジョンをしっかりと持ち、長期的な視点から自分のキャリアを構築することです。チャレンジの気持ちを持ちながら、自己啓発を日々行うことが必要です。\n\nこのような”キャリア自律”が注目されるようになった背景をセミナー深掘りしていきます！', metadata={'document_id': 1, 'title': 'ChatGPTのイベント参加ができるのか？検証イベント', 'url': 'https://bpo-protea.commmune.com/view/event/6432'}),
 Document(page_content='こちらの投稿で紹介している内容を皆さんと会話しながら紐解きます！\u3000https://bpo-protea.commmune.com/view/pos

In [4]:
# jsonファイルの読み込み
def load_json():
    # テキストデータの読み込み
    with open("../data/baseDocs.json", "r") as f:
        # jsonファイルを読み込んで、辞書型に変換する
        data = json.load(f)
        base_docs = [Document(page_content=text["page_content"], metadata=text["metadata"]) for text in data]

    # 指示文の読み込み
    with open("../data/instructions.json", "r") as f:
        # jsonファイルを読み込んで、辞書型に変換する
        instructions = json.load(f)

    return base_docs, instructions

In [5]:
# モデルの初期化
def initialize_model():
    chat = AzureChatOpenAI(
        temperature=0,
        openai_api_key=settings.AZURE_OPENAI_API_KEY,
        openai_api_base=settings.AZURE_OPENAI_API_ENDPOINT,
        deployment_name=settings.AZURE_OPENAI_API_DEPLOYMENT_NAME,
        openai_api_version=settings.AZURE_OPENAI_API_VERSION,
        openai_api_type="azure",
        max_tokens=1000,
    )

    embeddings = OpenAIEmbeddings(
        openai_api_key=settings.AZURE_OPENAI_API_KEY,
        openai_api_base=settings.AZURE_OPENAI_API_ENDPOINT,
        openai_api_version=settings.AZURE_OPENAI_API_VERSION,
        openai_api_type="azure",
        model=settings.EMBEDDING_MODEL_NAME,
        chunk_size=1
    )
    return chat, embeddings

In [6]:
# ドキュメント解析
def analyzeDocuments(chat, base_docs, instruction):
    qa_chain = load_qa_chain(chat, chain_type="map_reduce", verbose=False)
    qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

    docs = [
        Document(
            page_content=qa_document_chain.run(input_document=doc.page_content, question=instruction),
            metadata=doc.metadata
        ) for doc in tqdm(base_docs)
    ]
    return docs

In [9]:
# ベクトルストアに保存
def saveVector(embeddings, docs, table_name):

    # Qdrant
    Qdrant.from_documents(
        documents=docs,
        embedding=embeddings,
        url=settings.QDRANT_URL,
        prefer_grpc=True,
        collection_name=table_name,
    )

    # PGVector
    CONNECTION_STRING = PGVector.connection_string_from_db_params(
        driver="psycopg2",
        host="localhost",
        port=settings.PORT_PGVECTOR,
        database=settings.POSTGRES_DB,
        user=settings.POSTGRES_USER,
        password=settings.POSTGRES_PASSWORD,
    )

    PGVector.from_documents(
        embedding=embeddings,
        documents=docs,
        collection_name=table_name,
        connection_string=CONNECTION_STRING,
    )


In [ ]:
def main():
    try:
        # jsonファイルの読み込み
        base_docs, instructions = load_json()

        # モデルの初期化
        chat, embeddings = initialize_model()

        # 原文のまま
        saveVector(embeddings, base_docs, "original")

        # # 要約
        # for item in instructions:
        #     table_name, instruction = item.values()
        #     docs = analyzeDocuments(chat, base_docs, instruction)
        #     saveVector(embeddings, docs, table_name)

        # # 分割
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=300)
        # docs = text_splitter.split_documents(base_docs)
        # saveVector(embeddings, docs, "split")

    except Exception as e:
        print(e)

main()